# Computation of Dyadic Synchronization

### Import essential libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import pi #Used for some experimentation purposes
from scipy.fftpack import fft
import seaborn as sns
from tslearn.metrics import dtw, dtw_path
import scipy.signal
from cmath import rect,pi
import math
from functools import reduce

### Read the .csv file created

In [19]:
# Enter the path of the csv file
path='./DataFiles/KeypointData/Brahms/Original/BrahmsConcertPart1_File1.csv'
# Convert it into a dataframe
data=pd.read_csv(path,index_col=0)

In [20]:
data

,image_id,c1_x,c1_y,c2_x,c2_y,c3_x,c3_y,c4_x,c4_y,c5_x,c5_y,t
0,0,115.195282,218.366531,204.472473,222.705566,350.922516,217.265533,458.661499,211.814560,541.797852,213.232773,0.000000
1,1,114.755905,219.640488,203.437317,221.983002,350.936584,217.110672,457.244049,211.501221,540.319763,214.812668,0.033333
2,2,114.354630,218.721878,203.958145,221.870529,350.785492,217.054306,458.187622,209.169586,537.758301,216.730225,0.066667
3,3,114.560234,218.824707,204.310364,222.105301,350.882629,216.902542,459.209259,209.548096,535.944641,217.899170,0.100000
4,4,115.184525,217.648193,204.625656,222.507294,350.813690,217.328583,459.625763,206.986847,532.380676,217.271835,0.133333
...,...,...,...,...,...,...,...,...,...,...,...,...
535,535,123.022469,218.712280,180.852829,221.756531,340.252747,217.798874,463.574524,210.814514,494.895966,224.216415,17.833333
536,536,122.934143,220.853806,181.055908,221.613556,340.827820,218.825607,463.854553,211.056000,497.899414,223.047668,17.866667
537,537,124.647224,220.657394,180.037781,219.822510,340.567108,218.754257,463.772797,211.124573,497.946289,223.056412,17.900000
538,538,123.144516,220.862366,181.621841,221.701218,340.954651,218.590576,463.152100,210.737396,500.416412,221.713623,17.933333


## Feature extraction block: Compute distance between each time step

In [22]:
def distance(a,b,c):
    hello=a.copy()
    cold= str(a[b].name[0:2])+'_d'
    for i,rows in a.iterrows():
        if(i==0):
            hello.loc[i, cold] = 0
        else:
            hello.loc[i, cold] = math.sqrt((hello.loc[i,str(a[b].name[0:2])+'_x'] - hello.loc[i-1,str(a[b].name[0:2])+'_x'])**2 + (hello.loc[i,str(a[b].name[0:2])+'_y'] - hello.loc[i-1,str(a[b].name[0:2])+'_y'])**2)
            hello.loc[i,'t'] = hello.loc[i-1,'t'] + t_interval
    return hello


data1=distance(data,'c1_x','c1_y')
data2=distance(data1,'c2_x','c2_y')
data3=distance(data2,'c3_x','c3_y')
data4=distance(data3,'c4_x','c4_y')
dataFinal=distance(data4,'c5_x','c5_y')


# dataFinal[['c1_d','c2_d','c3_d','c4_d','c5_d']]=dataFinal[['c1_d','c2_d','c3_d','c4_d','c5_d']].apply(lambda x: bandPassFilter(x,30,0.1,3,4)).round(2)

dataFinal

,image_id,c1_x,c1_y,c2_x,c2_y,c3_x,c3_y,c4_x,c4_y,c5_x,c5_y,t,c1_d,c2_d,c3_d,c4_d,c5_d
0,0,115.195282,218.366531,204.472473,222.705566,350.922516,217.265533,458.661499,211.814560,541.797852,213.232773,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,114.755905,219.640488,203.437317,221.983002,350.936584,217.110672,457.244049,211.501221,540.319763,214.812668,0.033333,1.347597,1.262398,0.155499,1.451670,2.163519
2,2,114.354630,218.721878,203.958145,221.870529,350.785492,217.054306,458.187622,209.169586,537.758301,216.730225,0.066667,1.002430,0.532834,0.161264,2.515323,3.199705
3,3,114.560234,218.824707,204.310364,222.105301,350.882629,216.902542,459.209259,209.548096,535.944641,217.899170,0.100000,0.229885,0.423292,0.180189,1.089501,2.157729
4,4,115.184525,217.648193,204.625656,222.507294,350.813690,217.328583,459.625763,206.986847,532.380676,217.271835,0.133333,1.331887,0.510889,0.431582,2.594893,3.618756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,535,123.022469,218.712280,180.852829,221.756531,340.252747,217.798874,463.574524,210.814514,494.895966,224.216415,17.833333,2.474187,1.780972,0.452139,0.686623,1.325646
536,536,122.934143,220.853806,181.055908,221.613556,340.827820,218.825607,463.854553,211.056000,497.899414,223.047668,17.866667,2.143346,0.248361,1.176814,0.369772,3.222836
537,537,124.647224,220.657394,180.037781,219.822510,340.567108,218.754257,463.772797,211.124573,497.946289,223.056412,17.900000,1.724304,2.060201,0.270299,0.106707,0.047683
538,538,123.144516,220.862366,181.621841,221.701218,340.954651,218.590576,463.152100,210.737396,500.416412,221.713623,17.933333,1.516623,2.457395,0.420691,0.731553,2.811510


#### Create duplicate of dataFinal

In [27]:
df=dataFinal.copy()

In [28]:
df

,image_id,c1_x,c1_y,c2_x,c2_y,c3_x,c3_y,c4_x,c4_y,c5_x,c5_y,t,c1_d,c2_d,c3_d,c4_d,c5_d
0,0,115.195282,218.366531,204.472473,222.705566,350.922516,217.265533,458.661499,211.814560,541.797852,213.232773,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,114.755905,219.640488,203.437317,221.983002,350.936584,217.110672,457.244049,211.501221,540.319763,214.812668,0.033333,1.347597,1.262398,0.155499,1.451670,2.163519
2,2,114.354630,218.721878,203.958145,221.870529,350.785492,217.054306,458.187622,209.169586,537.758301,216.730225,0.066667,1.002430,0.532834,0.161264,2.515323,3.199705
3,3,114.560234,218.824707,204.310364,222.105301,350.882629,216.902542,459.209259,209.548096,535.944641,217.899170,0.100000,0.229885,0.423292,0.180189,1.089501,2.157729
4,4,115.184525,217.648193,204.625656,222.507294,350.813690,217.328583,459.625763,206.986847,532.380676,217.271835,0.133333,1.331887,0.510889,0.431582,2.594893,3.618756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,535,123.022469,218.712280,180.852829,221.756531,340.252747,217.798874,463.574524,210.814514,494.895966,224.216415,17.833333,2.474187,1.780972,0.452139,0.686623,1.325646
536,536,122.934143,220.853806,181.055908,221.613556,340.827820,218.825607,463.854553,211.056000,497.899414,223.047668,17.866667,2.143346,0.248361,1.176814,0.369772,3.222836
537,537,124.647224,220.657394,180.037781,219.822510,340.567108,218.754257,463.772797,211.124573,497.946289,223.056412,17.900000,1.724304,2.060201,0.270299,0.106707,0.047683
538,538,123.144516,220.862366,181.621841,221.701218,340.954651,218.590576,463.152100,210.737396,500.416412,221.713623,17.933333,1.516623,2.457395,0.420691,0.731553,2.811510


In [31]:
first, second, third = np.split(df, [int((1/3)*len(df)), int((2/3)*len(df))])
first.reset_index(drop=True,inplace=True)
second.reset_index(drop=True,inplace=True)
third.reset_index(drop=True,inplace=True)

# Measurement of entrainment block

## Functions

##### Functions for getting windowed data

In [13]:
def getWindowData(a):
    pct={}
    for i in range(count):
        pct['window'+str(i)]=d['group'+str(i)][[a,'t']]
    return pct

##### Function for applying FFT on the data

In [14]:
def getFFT(a):
    N=len(a)
    sample_rate=30
    waveform=np.array(a)
    freq_data = fft(waveform)
    y = (2/N) * np.abs (freq_data [0:np.int (N/2)])
    y_angle= np.angle(freq_data,deg=False)
    y_ampl= np.abs(freq_data)
    df_pct_ang_amp=pd.DataFrame({'Angle':y_angle,'Amplitude':y_ampl})
    return df_pct_ang_amp

##### Function for storing FFT results in a dictionary

In [15]:
def getFFTDict(pct,pct_data):
    #person1/person2 are participants
    #person1_aa is participant angle and amplitude data of person1 and likewise for person2_aa
    dict_paa={}
    for i in range(count):
        df=getFFT(pct_data['window'+str(i)][pct+'_d'])
        dict_paa['window'+str(i)]=df
    return dict_paa

##### Function for computing the relative phase angles between the participant data 

In [16]:
def rel_angle(l1,l2):
    rel_angledict={}
    for i in range(count):
        rel_anglex=l1['window'+str(i)]['Angle']-l2['window'+str(i)]['Angle']
        rel_angledict['window'+str(i)]=rel_anglex
    return rel_angledict

##### Functionf or computing the dyadic synchronization between all participants in an automated manner. It sets the limtis to 1:51 for a window size of 150 and 1:11 for a window size of 30.

In [17]:
def Automated():
    endLimit=int((windowSize/3)+1)
    for i in range(5):
        for j in range(5):
            participant1='c'+str(i+1)
            participant2='c'+str(j+1)
            if ((participant1!=participant2)&(j>i)):
                person1=getWindowData(participant1+'_d')
                person2=getWindowData(participant2+'_d')
                person1_aa=getFFTDict(participant1,person1)
                person2_aa=getFFTDict(participant2,person2)

                x=rel_angle(person1_aa,person2_aa)

                plv_results={}

                for k in range(count_for_window):
                    my_list = [elem[k] for elem in x.values()]
                    rel_amp=[1]*len(my_list)
                    rel_sum = [x for x in map(rect,rel_amp,my_list)]
                    plv_results[str(k)]=abs(sum(rel_sum))/(len(rel_sum))

                plv_results1=list(plv_results.values())
                print((sum(plv_results1[1:endLimit])/len(plv_results1[1:endLimit])))
#                 print((sum(plv_results1[1:26])/len(plv_results1[1:26])))
    print("Time Under Observation:",df['t'].max())
    print("Count for window:",count_for_window)
    print("Window Size",windowSize)
    print("Number of samples taken for averaging:",endLimit-1)
    #             print("Dyadic Synchronization between ",participant1," and ",participant2,":",(sum(plv_results1[1:endLimit])/len(plv_results1[1:endLimit])))

## PLV Computation: Entire phrase

In [29]:
windowSizeFlag="off"

d={}
if windowSizeFlag=="on":
    windowSize=30
else:
    windowSize=150
windowBegin=0
windowEnd=windowSize-1
if(windowSize==150):
    shift=15
else:
    shift=5
count=1
count_start=windowBegin+windowEnd+1
count_for_window=count_start
while(count_start!=len(df)):

    count+=1
    count_start+=shift

for i in range(count):
    df1=df.loc[windowBegin:windowEnd]
    d["group"+str(i)]=df1
    windowBegin+=shift
    windowEnd+=shift

In [30]:
Automated()

C:\Users\casap\AppData\Local\Temp\ipykernel_12628\1095152042.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = (2/N) * np.abs (freq_data [0:np.int (N/2)])
C:\Users\casap\AppData\Local\Temp\ipykernel_12628\1095152042.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the 

0.3982595868073993
0.4288349298092472
0.41313119299052536
0.40676212817330987
0.3635231861688693
0.4326337912813979
0.35602180449429477
0.4388060282456943
0.42563200727693706
0.39381012626786754
Time Under Observation: 17.96666666666671
Count for window: 150
Window Size 150
Number of samples taken for averaging: 50


C:\Users\casap\AppData\Local\Temp\ipykernel_12628\1095152042.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = (2/N) * np.abs (freq_data [0:np.int (N/2)])
C:\Users\casap\AppData\Local\Temp\ipykernel_12628\1095152042.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the 

## PLV Computation: Each part (start, middle, end)

##### First part 

In [35]:
df=first
windowSizeFlag="on"

d={}
if windowSizeFlag=="on":
    windowSize=30
else:
    windowSize=150
windowBegin=0
windowEnd=windowSize-1
if(windowSize==150):
    shift=15
else:
    shift=5
count=1
count_start=windowBegin+windowEnd+1
count_for_window=count_start
while(count_start!=len(df)):

    count+=1
    count_start+=shift

for i in range(count):
    df1=df.loc[windowBegin:windowEnd]
    d["group"+str(i)]=df1
    windowBegin+=shift
    windowEnd+=shift
    
Automated()

C:\Users\casap\AppData\Local\Temp\ipykernel_12628\1095152042.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = (2/N) * np.abs (freq_data [0:np.int (N/2)])
C:\Users\casap\AppData\Local\Temp\ipykernel_12628\1095152042.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the 

0.23331114732860286
0.20868185713750034
0.29290402373310426
0.3162131574803173
0.3174161543242997
0.3488093566185639


C:\Users\casap\AppData\Local\Temp\ipykernel_12628\1095152042.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = (2/N) * np.abs (freq_data [0:np.int (N/2)])
C:\Users\casap\AppData\Local\Temp\ipykernel_12628\1095152042.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the 

0.326445701552487
0.36553510169705267
0.36187666172548716
0.39845079403730166
Time Under Observation: 5.966666666666649
Count for window: 30
Window Size 30
Number of samples taken for averaging: 10


##### Second part

In [34]:
df=second
windowSizeFlag="on"


d={}
if windowSizeFlag=="on":
    windowSize=30
else:
    windowSize=150
windowBegin=0
windowEnd=windowSize-1
if(windowSize==150):
    shift=15
else:
    shift=5
count=1
count_start=windowBegin+windowEnd+1
count_for_window=count_start
while(count_start!=len(df)):

    count+=1
    count_start+=shift

for i in range(count):
    df1=df.loc[windowBegin:windowEnd]
    d["group"+str(i)]=df1
    windowBegin+=shift
    windowEnd+=shift
    
Automated()

C:\Users\casap\AppData\Local\Temp\ipykernel_12628\1095152042.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = (2/N) * np.abs (freq_data [0:np.int (N/2)])
C:\Users\casap\AppData\Local\Temp\ipykernel_12628\1095152042.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the 

0.32672854295305076
0.30558624657297023
0.39133048973830004
0.31893807947026753
0.38102286886255665
0.3433407184046052
0.3265136475722817
0.42471024826158715
0.37202646611547296
0.32613602336880326
Time Under Observation: 11.966666666666628
Count for window: 30
Window Size 30
Number of samples taken for averaging: 10


C:\Users\casap\AppData\Local\Temp\ipykernel_12628\1095152042.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = (2/N) * np.abs (freq_data [0:np.int (N/2)])
C:\Users\casap\AppData\Local\Temp\ipykernel_12628\1095152042.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the 

##### Third part

In [33]:
df=third
windowSizeFlag="on"

d={}
if windowSizeFlag=="on":
    windowSize=30
else:
    windowSize=150
windowBegin=0
windowEnd=windowSize-1
if(windowSize==150):
    shift=15
else:
    shift=5
count=1
count_start=windowBegin+windowEnd+1
count_for_window=count_start
while(count_start!=len(df)):

    count+=1
    count_start+=shift

for i in range(count):
    df1=df.loc[windowBegin:windowEnd]
    d["group"+str(i)]=df1
    windowBegin+=shift
    windowEnd+=shift
    
Automated()

C:\Users\casap\AppData\Local\Temp\ipykernel_12628\1095152042.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = (2/N) * np.abs (freq_data [0:np.int (N/2)])
C:\Users\casap\AppData\Local\Temp\ipykernel_12628\1095152042.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the 

0.2862062057874391
0.30280260628428934
0.351818972354674
0.3202109209541233
0.25832047343551967
0.3318712448760673


C:\Users\casap\AppData\Local\Temp\ipykernel_12628\1095152042.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = (2/N) * np.abs (freq_data [0:np.int (N/2)])
C:\Users\casap\AppData\Local\Temp\ipykernel_12628\1095152042.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the 

0.28408020372681697
0.23309547154849936
0.31764656583080714
0.30452442039658223
Time Under Observation: 17.96666666666671
Count for window: 30
Window Size 30
Number of samples taken for averaging: 10


C:\Users\casap\AppData\Local\Temp\ipykernel_12628\1095152042.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = (2/N) * np.abs (freq_data [0:np.int (N/2)])
C:\Users\casap\AppData\Local\Temp\ipykernel_12628\1095152042.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the 